In [ ]:
!pip install anytree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import json
import anytree
from anytree import AnyNode,RenderTree
import pandas as pd
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import string
import re
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('nltk')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np
from xgboost import XGBClassifier

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading nltk: Package 'nltk' not found in index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# **Jaccard Similarity using Custom Ontology**

In [ ]:
file=open("/Ontologies/ontology.json")
data=json.load(file)



The below code snippet defines two functions: `get_synonyms` and `createTree`.

### Function `get_synonyms(data,component)`
- Parameters:
  - `data`: The input data, expected to be a list.
  - `component`: The component to search for in the data.
- Returns: The synonyms corresponding to the given component in the data.

The function performs the following steps:
1. It iterates over each item in the data.
2. If the second element of the item matches the given component, it returns the third element (synonyms).
3. If no match is found, it returns `None`.

### Function `createTree(data)`
- Parameters:
  - `data`: The input data, expected to be a list.
- Returns: The root node of the created tree.

The function performs the following steps:
1. It initializes a set `parentsList` to store unique parent values.
2. It initializes an empty dictionary `dictionaryTree` to store the tree structure.
3. It iterates over each item in the data.
   - It adds the first element of the item to the `parentsList` set.
4. It iterates over each parent in the `parentsList`.
   - It initializes an empty list as the value for the parent in the `dictionaryTree`.
   - It iterates over each value in the data.
     - If the first element of the value matches the current parent, it appends the second element to the list of the parent's values in the `dictionaryTree`.
5. It creates a root node with `id="islam"`, empty `synonymList`, level `0`, and empty `parentId`.
6. It initializes a list `currentList` with `"islam"`.
7. It enters a while loop until the `dictionaryTree` is empty.
   - It initializes an empty list `tempList`.
   - It iterates over each current in the `currentList`.
     - If the current exists as a key in the `dictionaryTree`:
       - It iterates over each child in the values of the current in the `dictionaryTree`.
         - It appends the child to the `tempList`.
         - It searches for the parent node with id equal to the current in the existing tree using `anytree.search.find`.
         - It creates a new node with the child as the id, the found parent as the parent, synonyms obtained from `get_synonyms`, level incremented by 1, and the current as the parentId.
         - It updates the `tempCurrent` variable with the current.
       - It removes the current key from the `dictionaryTree`.
   - It clears the `currentList` and copies the elements from the `tempList`.
8. It returns the root node of the created tree.




In [ ]:
def get_synonyms(data,component):
    for item in data:
        if item[1]==component:
            return item[2]
def createTree(data):
    parentsList=set()
    dictionaryTree=dict()
    for item in data:
        parentsList.add(item[0])
    
    for item in parentsList:
        dictionaryTree[item]=[]
        for value in data:
            if value[0]==item:
                dictionaryTree[item].append(value[1])
  
    currentList=['islam']
    tempCurrent=None
    parent=AnyNode(id="islam",synonymList=[],level=0,parentId="")
    while len(dictionaryTree)>0:
        tempList=[]
        for current in currentList:
            if current in dictionaryTree.keys():
                for child in dictionaryTree[current]:
                    tempList.append(child)
                    tempNode=anytree.search.find(parent,lambda node:node.id==current)
                    node=AnyNode(id=child,parent=tempNode,synonymList=get_synonyms(data,child),level=(tempNode.level)+1,parentId=current)
                    tempCurrent=current
                del dictionaryTree[current]
        currentList.clear()
        currentList=tempList.copy()
    return parent



The below code snippet performs the following steps:

1. It initializes an empty list `finalData` to store the extracted ontology data.
2. It iterates over each item in the `data`.
   - It checks if the string "http://www.semanticweb.org/lenovo/ontologies/2022/8/untitled-ontology-2" is present in the '@id' key of the item.
   - It checks if the string "http://www.w3.org/2002/07/owl#Class" is present in the '@type' key of the item.
   - It checks if the key 'http://www.w3.org/2000/01/rdf-schema#subClassOf' is present in the item.
   - If all the conditions are satisfied, it extracts the component and parent values from the item and creates a list of synonyms.
   - It appends the extracted data as a list [parent, component, synonymList] to the `finalData` list.
3. It calls the `createTree` function with the `finalData` list to create the main ontology tree, assigning the root node to `MainTreeOntology`.
4. It iterates over the nodes in the ontology tree using `RenderTree` from the `anytree` library.
   - It prints the node's id with appropriate indentation based on the tree structure.




In [ ]:
finalData=[]
for item in data:
    if "http://www.semanticweb.org/lenovo/ontologies/2022/8/untitled-ontology-2" in str(item['@id']):
        if "http://www.w3.org/2002/07/owl#Class" in str(item['@type'][0]):
            if "http://www.w3.org/2000/01/rdf-schema#subClassOf" in item.keys():
                componentArray=str.split(str(item['@id']),'#')
                component=componentArray[1]
                parentArray=str.split(str(item['http://www.w3.org/2000/01/rdf-schema#subClassOf'][0]),'#')
                parent=parentArray[1]
                parent=parent[:len(parent)-2]
                if 'http://www.w3.org/2000/01/rdf-schema#label' in item.keys():
                   synonymList=[]
                   for synonym in item['http://www.w3.org/2000/01/rdf-schema#label']:
                        synonymList.append(str(synonym['@value']))
                # appendToFile(parent+":"+component+":Synonyms="+str(synonymList))  
                finalData.append([parent,component,synonymList]) 
                synonymList=[]
                

MainTreeOntology=createTree(finalData)
for pre, fill, node in RenderTree(MainTreeOntology):
    print("%s%s" % (pre, node.id))

islam
└── pillars
    ├── component
    │   ├── ablution_component
    │   │   ├── masah_of_head
    │   │   ├── masah_of_neck
    │   │   ├── rinse_mouth
    │   │   ├── wash_face
    │   │   ├── wash_feet_till_ankle
    │   │   ├── wash_hands
    │   │   ├── wash_hands_till_elbow
    │   │   └── wash_nostrils
    │   ├── ghusal_component
    │   │   ├── clean_body
    │   │   ├── clean_whole_body
    │   │   ├── rinse_nostrils
    │   │   ├── rinse_the_mouth
    │   │   └── wudu
    │   ├── kiblah
    │   ├── prayer_component
    │   │   ├── bowing
    │   │   ├── fatiha
    │   │   ├── intention
    │   │   ├── prostration
    │   │   ├── qadah
    │   │   ├── qauma
    │   │   ├── recitation
    │   │   ├── salaam
    │   │   ├── sitting
    │   │   ├── standing
    │   │   ├── standing_after_bow
    │   │   └── takbir
    │   └── tayammum_component
    │       ├── wipe_face
    │       └── wipe_hands
    ├── fasting
    │   ├── farz_fasting
    │   │   └── ramazan
    │   ├── nafi

In [ ]:
def lemmatizer(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
def preprocess_text(df):
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
  df['text'] = df['text'].apply(lambda x: x.split())
  stop_words = set(stopwords.words('english'))
  df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])
  df['text']=df['text'].apply(lambda x:lemmatizer(x))
  df['text'] = df['text'].apply(lambda x: ' '.join(x))
  return df['text']

def preprocess_text2(df):
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
  df['text'] = df['text'].apply(lambda x: x.split())
  stop_words = set(stopwords.words('english'))
  df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])
  stemmer = PorterStemmer()
  df['text'] = df['text'].apply(lambda x: [stemmer.stem(word) for word in x])
  df['text'] = df['text'].apply(lambda x: ' '.join(x))
  return df['text']

In [ ]:
df = pd.read_csv("/Data/train_data.csv")
df['label'] = df['label'].replace({"hate":0,"nothate":1})

In [ ]:
df['text'] = preprocess_text(df)
sentences = []

for value in df['text']:
  value = str(value)
  sentences.append(value.strip("[]'"))
i = 0
entities = []
word = ""
for x in range(len(df['text'])):
  temp = nltk.word_tokenize(sentences[x])
  text = nltk.pos_tag(temp)
  for bigram in text:
      word = word + " " + bigram[0]
  entities.append(word)
  word = ""



The below code snippet defines several functions: `isEmpty`, `getPillar`, and `getClassificationJaccard`.

### Function `isEmpty(dictionary)`
- Parameter:
  - `dictionary`: The input dictionary to check for emptiness.
- Returns: A boolean indicating whether the dictionary is empty.

The function performs the following steps:
1. It initializes a boolean variable `empty` as `True`.
2. It iterates over each key in the dictionary.
   - If any key has a value greater than 0, it updates `empty` to `False` and immediately returns `empty`.
3. If the loop completes without finding any non-zero values, it returns `empty`.

### Function `getPillar(nodeId)`
- Parameter:
  - `nodeId`: The id of the node to find its corresponding pillar.
- Returns: The id of the pillar node for the given nodeId.

The function performs the following steps:
1. It searches for the node with id equal to `nodeId` in the `MainTreeOntology`.
2. It initializes a variable `parent` with the found node.
3. It iterates from 0 to `nodeToFind.level-2`.
   - It updates `parent` to its parent node in each iteration.
4. It returns the id of the parent node.

### Function `getClassificationJaccard(toCheck, counter)`
- Parameters:
  - `toCheck`: The list of words to check for classification.
  - `counter`: A counter value.
- Returns: The label of the pillar node based on the classification Jaccard score.

The function performs the following steps:
1. It initializes an empty dictionary `document_words` to store the counts of words from `toCheck`.
2. It iterates over each word in `toCheck` and initializes its count in `document_words` to 0.
3. It iterates over each word in `toCheck`.
   - It searches for the word in the `MainTreeOntology`.
     - If found, it increments the count of the word in `document_words`.
     - If not found, it searches for the word in the synonyms of nodes in the `MainTreeOntology`.
       - If found, it adds the word as a key in `document_words` and increments its count.
4. It removes the keys from `document_words` with a count of 0 and removes the keys `'islam'` and `'pillars'` if present.
5. It calculates the `totalLevels` as the maximum level of nodes in the `MainTreeOntology`.
6. It initializes an empty list `connections` to store the connections between nodes in `document_words`.
7. While `document_words` is not empty:
   - It iterates over the levels from 0 to `totalLevels`.
     - It finds all nodes in the `MainTreeOntology` with a level equal to the current level.
     - For each node, if the node's id is present in `document_words` and its count is greater than 0:
       - It adds the node's id and level to `connections`.
       - It decrements the count of the node in `document_words`.
8. It initializes an empty dictionary `fragments` to store the fragments of nodes and their connections.
9. It iterates over the connections and populates `fragments` based on the level of the connection.
10. It iterates over the connections again and populates `fragments` with connections to the pillars.
11. It checks if the pillars are present in `fragments`, and if not,it adds them to fragments with empty connection lists.
12. It opens a file with a filename based on the counter value.
13. It writes the connections in fragments to the file.
14. It calculates the scores for each fragment based on the number of connections and the union of words in toCheck.
15. It initializes variables max and label to track the maximum score and the corresponding label.
16. It compares each fragment's score to the current maximum and updates max and label accordingly.
17. It returns the label as the final classification label based on the highest score.


In [ ]:
def isEmpty(dictionary):
    empty=True
    for key in dictionary.keys():
        if dictionary[key]>0:
            empty=False
            return empty
    return empty
def getPillar(nodeId):
    nodeToFind=anytree.search.find(MainTreeOntology,lambda node:nodeId==node.id)
    parent=nodeToFind
    for i in range(0,nodeToFind.level-2):
        parent=parent.parent
    return parent.id
def getClassificationJaccard(toCheck,counter):
    document_words=dict()
    for word in toCheck:
        document_words[word]=0

    for word in toCheck:
        tempNode=anytree.search.find(MainTreeOntology,lambda node:node.id==word)
        if tempNode is not None:
            document_words[word]+=1
        else:
            tempNode=anytree.search.find(MainTreeOntology,lambda node:word in node.synonymList)
            if tempNode is not None:
                if tempNode.id not in document_words.keys():
                    document_words[tempNode.id]=1
                else:
                    document_words[tempNode.id]+=1
    toDelete=[]
    for key in document_words.keys():
        if document_words[key]==0:
            toDelete.append(key)
    for key in toDelete:
        del document_words[key]
    if "islam" in document_words.keys():
        del document_words['islam']
    if "pillars" in document_words.keys():
        del document_words['pillars']
    totalLevels=0
    for pre, fill, node in RenderTree(MainTreeOntology):
        if node.level>totalLevels:
            totalLevels=node.level
    totalLevels
    connections=[]
    while not isEmpty(document_words):
        for i in range(0,totalLevels+1):
            nodes=anytree.search.findall(MainTreeOntology,filter_=lambda node: node.level==i)
            for node in nodes:
                if node.id in document_words.keys():
                    if document_words[node.id]>0:
                        connections.append([node.id,node.level])
                        document_words[node.id]-=1
    fragments=dict()
    for item in connections:
        if item[1]==2:
            if item[0] in fragments.keys():
                fragments[item[0]][0]+=1
            else:
                fragments[item[0]]=[1,[]]
    for item in connections:
        if item[1]>2:
            pillar=getPillar(item[0])
            if pillar in fragments.keys():
                fragments[pillar][1].append(item)
            else:
                fragments[pillar]=[0,[]]
                fragments[pillar][1].append(item)

    pillars=["fasting","namaz","shahadah","hajj","component","zakat"]
    for pillar in pillars:
        if pillar not in fragments.keys():
            fragments[pillar]=[0,[]]
    filename="r_nr_ontological_features\\"+str(counter)
    file=open('%s.txt' % filename,"w",encoding='utf-8')
    for fragment in fragments.keys():
        if len(fragments[fragment][1])>0:
            for word in fragments[fragment][1]:

                file.write(str(word[0])+"\n")
    scores=dict()
    # print(fragments)
    for key in fragments.keys():
        scores[key]=fragments[key][0]+len(fragments[key][1])
    union=len(toCheck)
    for keys in scores.keys():
        scores[keys]=scores[keys]/union
    max=0
    label=""
    for keys in scores.keys():
        if scores[keys]>max:
            max=scores[keys]
            label=keys
    return label

In [ ]:
bow = []
for x in entities:
  bow.append(x.strip().split(" "))
listOfLabels=[]
i=1
for item in bow:
    label=getClassificationJaccard(item,i)
    i+=1
    listOfLabels.append(label)
    
for i in range(0,len(listOfLabels)):#"hate":0,"nothate":1
    if listOfLabels[i] == "":
      listOfLabels[i] = 1
    else:
      listOfLabels[i] = 0

print("Accuracy: "+str(accuracy_score(df['label'],listOfLabels)))
cm = confusion_matrix(df['label'],listOfLabels)
print(cm)
print(classification_report(df['label'],listOfLabels))

Accuracy: 0.4437416777629827
[[  54 1571]
 [ 100 1279]]
              precision    recall  f1-score   support

           0       0.35      0.03      0.06      1625
           1       0.45      0.93      0.60      1379

    accuracy                           0.44      3004
   macro avg       0.40      0.48      0.33      3004
weighted avg       0.40      0.44      0.31      3004



# **MultinomialNB & Logistic Regression**

In [ ]:
df = pd.read_csv("/Data/train_data.csv")
df['label'] = df['label'].replace({"hate":0,"nothate":1})
df['text'] = preprocess_text2(df)

df2 = pd.read_csv("/Data/test_data.csv")
df2['label'] = df2['label'].replace({"hate":0,"nothate":1})
df2['text'] = preprocess_text2(df2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
data_tfidf = vec.fit_transform(df['text'])
print(vec.get_feature_names_out())
print(data_tfidf[0])
print(data_tfidf.shape)
test_data = vec.transform(df2['text'])

['abb' 'abhor' 'abhorr' ... 'zoo' 'zulpan' 'zwemer']
  (0, 2507)	0.4006738904780003
  (0, 770)	0.4580337503288457
  (0, 1526)	0.16581950382312524
  (0, 2377)	0.4538587923756214
  (0, 2006)	0.39473454552860404
  (0, 772)	0.49027160263514946
(3004, 2618)


In [ ]:
# "hate":0,"nothate":1
lr=LogisticRegression(penalty='l2',max_iter=1000,C=1,random_state=36,class_weight={1: 1.12, 0:1},solver="lbfgs")
lr_bow=lr.fit(data_tfidf,df['label'])
print("Original Logistic Regression Score Training:",lr.score(data_tfidf,df['label']))
print("Original Logistic Regression Score Testing:",lr.score(test_data,df2['label']))

print(confusion_matrix(df2['label'],lr_bow.predict(test_data)))
print(classification_report(df2['label'],lr_bow.predict(test_data)),end='\n\n')

gn2 = MultinomialNB(class_prior=[1.15,0.9])
gn2_bow=gn2.fit(data_tfidf,df['label'])

print("Original MultinomialNB Score Training:",gn2.score(data_tfidf,df['label']))
print("Original MultinomialNB Score Testing:",gn2.score(test_data,df2['label']))

print(confusion_matrix(df2['label'],gn2_bow.predict(test_data)))
print(classification_report(df2['label'],gn2_bow.predict(test_data)),end='\n\n')

Original Logistic Regression Score Training: 0.9101198402130493
Original Logistic Regression Score Testing: 0.8622754491017964
[[166  23]
 [ 23 122]]
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       189
           1       0.84      0.84      0.84       145

    accuracy                           0.86       334
   macro avg       0.86      0.86      0.86       334
weighted avg       0.86      0.86      0.86       334


Original MultinomialNB Score Training: 0.9067909454061251
Original MultinomialNB Score Testing: 0.874251497005988
[[171  18]
 [ 24 121]]
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       189
           1       0.87      0.83      0.85       145

    accuracy                           0.87       334
   macro avg       0.87      0.87      0.87       334
weighted avg       0.87      0.87      0.87       334


